In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib

import matplotlib.pyplot as plt
from ska_pst_stat import Statistics
from ska_pst_stat.utility import Hdf5FileGenerator, StatConfig

## Generate a test file

The following shows how to generate a test file that can then be used to display the statistics

First create a file path and file name

In [ ]:
output_file = pathlib.Path() / "test.h5"

# ensure the file doesn't exist before creating it
if output_file.exists():
    output_file.unlink()

Create an instance of a Hdf5FileGenerator. This is an example of a MID telescope scan

In [ ]:
generator = Hdf5FileGenerator(
    file_path=output_file,
    eb_id="eb-m001-20230921-245",
    telescope="SKAMid",
    scan_id=42,
    beam_id="1",
    config=StatConfig(nheap=16, ntime_bins=32, nbit=8, nchan=555),
)

Generate the HDF5 file

In [ ]:
generator.generate()

Assert the file now exists

In [ ]:
assert output_file.exists()

## Use STAT file to view statistics

Everything below now assumes that the file was generated by `STAT.CORE`

The first thing to do is load the statistics from the given file path.

In [ ]:
stats = Statistics.load_from_file(file_path=output_file)

Display the scalar header fields.

In [ ]:
stats.header

## View frequency averaged stats

View the frequency averaged stats for all frequencies

This returns a Pandas data frame with a multi-index key of polarisation and complex voltage dimension

In [ ]:
df = stats.frequency_averaged_stats
df

To view the data with RFI excised, use the `frequency_averaged_stats_rfi_excised` property (Note in this example there has been no RFI excision)

In [ ]:
df = stats.frequency_averaged_stats_rfi_excised
df

## View per channel level statistics

There are 6 properties that can be used to get channel level statistics; 3 for each polariation.  Within each polarisation you can get the combined complex data or you can select the real or imaginary data.

* pol_a_channel_stats
* pol_a_real_channel_stats
* pol_a_imag_channel_stats
* pol_b_channel_stats
* pol_b_real_channel_stats
* pol_b_imag_channel_stats

For example so view all of the polarisation A channel stats:

In [ ]:
df = stats.pol_a_channel_stats
df

or to view say only the real valued data from polarisation B:

In [ ]:
df = stats.pol_b_real_channel_stats
df

## Spectral power statistics

The STAT file includes the mean and max spectral power for each channel over the time the STAT file performs the calculations. This is recorded for each polarisation and can be accessed by the following properties

* pol_a_spectral_power
* pol_b_spectral_power

The following shows accessing both properties

In [ ]:
df = stats.pol_a_spectral_power
df

In [ ]:
df = stats.pol_b_spectral_power
df

Below is an example of using the above data frames can be to find which channel has the overall maximum power (such as ones that have RFI).

In this case Channel 409 has the maximum spectral power.

In [ ]:
df.iloc[df["Max"].idxmax()]  # type: ignore

## Histogram Data (non-rebinned)

There are 8 histogram data frames that can be extracted from the STAT file. These are made up of the permutations of different polarisations, real vs imag data, and all channels or just the channels that don't have RFI.

* pol_a_real_histogram
* pol_a_imag_histogram
* pol_a_real_histogram_rfi_excised
* pol_a_imag_histogram_rfi_excised
* pol_b_real_histogram
* pol_b_imag_histogram
* pol_b_real_histogram_rfi_excised
* pol_b_imag_histogram_rfi_excised

As an example the following is for imaginary component of polarisation A.

In [ ]:
df = stats.pol_a_imag_histogram
df

To plot the histogram data, use something like the following from Matplotlib

In [ ]:
plt.hist(df["Bin"], bins=256, weights=df["Count"])
plt.show()

## Histogram Data (Rebinned)

The data may also rebinned to a finer number of bins. There are also 1D histograms (as above) but there are 4 2D histograms; this is a combination of polarisation and all or just rfi excised data.

### Available 1D histograms

* pol_a_real_rebinned_histogram
* pol_a_imag_rebinned_histogram
* pol_a_real_rebinned_histogram_rfi_excised
* pol_a_imag_rebinned_histogram_rfi_excised
* pol_b_real_rebinned_histogram
* pol_b_imag_rebinned_histogram
* pol_b_real_rebinned_histogram_rfi_excised
* pol_b_imag_rebinned_histogram_rfi_excised

Plotting the above histograms are similar to the non-rebinned 1D histograms

### Available 2D histograms

* pol_a_rebinned_histogram2d
* pol_b_rebinned_histogram2d
* pol_a_rebinned_histogram2d_rfi_excised
* pol_b_rebinned_histogram2d_rfi_excised

An example of plotting a 2D histogram is as follows.  Note that in the 2D case we get a Numpy array not Pandas data frame

In [ ]:
hist2d = stats.pol_b_rebinned_histogram2d
hist2d

In [ ]:
plt.imshow(hist2d, origin="lower")
plt.show()

## Spectrograms

The STAT data also includes spectrogram data for both polarisations.  The following properties are available as spectrograms:

* pol_a_spectrogram
* pol_b_spectrogram

Below is an example of how to plot the spectrogram

In [ ]:
plt.imshow(stats.pol_a_spectrogram)
plt.show()

## Timeseries data

The STAT data provides a timeseries data. This is per temporal bin in the spectrogram that all the power of all frequencies are combined.  There are 4 timeseries data frames based on the 2 polarisations and all frequencies or just of the RFI excised data.

* pol_a_timeseries
* pol_b_timeseries
* pol_a_timeseries_rfi_excised
* pol_b_timeseries_rfi_excised

The timeseries data reports max, min, and mean power per temporal bin.

The following is an example of plotting the time series data. Note that there is an extra column of the `Time offset` that is used to give a time scale on the X axis.

In [ ]:
df = stats.pol_a_timeseries_rfi_excised
df

In [ ]:
plt.plot(df["Time offset"], df[["Max", "Min", "Mean"]])
plt.show()